Harris Economic Order Quantity (EOQ) 

`y` is the optimal order quantity

`m` is the units per month

`s` is the setup cost of an order

`c` is the unit price of items in the stock

In [1]:
from dataclasses import dataclass
import jax.numpy as jnp


def ooq(m: float, c: float, s: float) -> int:
    """
    Optimal Order Quantity.
    
    Economic order quantity function defined by Harris, 1990. 
    
    The number 240 is an "interest charge" per unit, per month, 
    corresponding to a yearly interest rate of 10 percent.
    
    :param x: dataclass representing a vector of model inputs. 
    :return: optimal order quantity
    """
    return jnp.sqrt(240 * m * s / c)

X = {'m': 100, 'c': 5, 's': 1}
y = ooq(X['m'], X['c'], X['s'])
print(y)

69.282036


## Local Sensitivity Methods
If we want to perform sensitivity analysis around some point of interest, then we are performing local analysis. This is in contrast to global analysis. 

These local methods include:
- One-at-a-Time
- Scenario Decomposition
- Partial Differentiation


## Sensitivity Analysis with Finite Differences
### One-at-a-Time (OAT) Methods

We assign a base case $x^0$ and a sensitivity case $x^+$ to the model 
inputs. We could also have an $x^-$ case, but that is not shown here.

For $x^+$, we increase each model input, one at a time, by 10% (this is an arbitrary choice and would change with the model). For an $x^-$ case, we could then **decrease** all inputs by 10%. 

Then, we compare the resulting $\Delta^+ y$ outputs (one for each changed parameter), where:

$$\Delta_i^+ y = g(x_i^+, x^0_{\neg i}) - y^0$$ 

So, in our example case, we might have:

$$\Delta_m^+ y = g(m^+, c^0, s^0) - y^0$$ 

In [28]:
 # Initial parameters                                                                                        
 x0 = {'m': 1230.0, 'c': 0.0135, 's': 2.15}                                                                    
 y0 = ooq(x0['m'], x0['c'], x0['s'])                                                      
                                                                                                             
 # Parameters with changes                                                                                   
 x_plus_m = {'m': 1353, 'c': 0.0135, 's': 2.15}                                                              
 x_plus_s = {'m': 1230, 'c': 0.01485, 's': 2.15}                                                             
 x_plus_c = {'m': 1230, 'c': 0.0135, 's': 2.365}                                                             
 x_plus = {'m': 1353, 'c': 0.01485, 's': 2.365}                                                              
                                                                                                             
 # Calculate changes in output                                                                               
 delta_y_m = ooq(x_plus_m['m'], x_plus_m['c'], x_plus_m['s']) - y0                        
 delta_y_s = ooq(x_plus_s['m'], x_plus_s['c'], x_plus_s['s']) - y0                        
 delta_y_c = ooq(x_plus_c['m'], x_plus_c['c'], x_plus_c['s']) - y0                        
                                                                                                             
 # Store deltas in a list                                                                                    
 deltas = [delta_y_m, delta_y_s, delta_y_c]         
 deltas = [delta.item() for delta in deltas]
                                                                                                             
 # Print results                                                                                             
 print("Base case:", y0)                                                                                     
 print("Changes in model output with respect to m, s, and c:\n", deltas)                                     
                                                                                

Base case: 6856.627
Changes in model output with respect to m, s, and c:
 [334.6640625, -319.08984375, 334.6640625]


This indicates that when `m` is increased by 10%, the model output increases
 by 335 units. When `s` is increased, the model output decreases by 319 
 units, and so on. 
 

**Question:** For a Particle Filter, we can consider $\beta$ and case counts as inputs, with the likelihood as the output. Do we have a "best case" and "worst case"? Maybe a "maximum case" (high beta, high case count) and a "minimum case" (no new cases)?


In [3]:
delta_y_plus = ooq(x_plus['m'], x_plus['c'], x_plus['s']) - y0
sum_of_delta = jnp.sum(jnp.array(deltas))

print("Change in y from x0 to x_plus:", delta_y_plus)
print("Sum of individual changes:", sum_of_delta)

Change in y from x0 to x_plus: 334.66406
Sum of individual changes: 350.23828


**Note that these changes are not equal**. This is due to the fact that this method of 
sensitivity analysis does not account for interaction effects between the input variables.

## Visualization
The standard way to visualize these changes is a **Tornado Diagram**. 

![tornado_diagram](./tornado_diagram.png)

## Scenario Decomposition
We start with a base case $x^0$, best case $x^+$, and worst case $x^-$.

**Problem**:
We could compare the outputs of y(x^+), y(x^0), y(x^-). However, we would not know what caused the changes from one to another. The following methods attempt to rectify this issue.

We can decompose $\Delta y = g(x^+) - g(x^0)$ by utilizing finite differences, where:

$$\delta y = 


In [3]:
# Initial parameters                                                                                        
x0 = {'m': 1230.0, 'c': 0.0135, 's': 2.15}                                                                    
y0 = ooq(x0['m'], x0['c'], x0['s'])                                                      
                                                                                                             
# Parameters with changes                                                                                   
x_plus_m = {'m': 1353, 'c': 0.0135, 's': 2.15}                                                              
x_plus_s = {'m': 1230, 'c': 0.01485, 's': 2.15}                                                             
x_plus_c = {'m': 1230, 'c': 0.0135, 's': 2.365}                                                             
x_plus = {'m': 1353, 'c': 0.01485, 's': 2.365}   
 

In [40]:
import itertools


# Helper function to call ooq with a dictionary of parameters
def g(params):
    return ooq(params['m'], params['c'], params['s'])


# Calculate phi recursively
def calculate_phi(x_plus, x0, g):
    all_keys = list(x_plus.keys())
    n = len(all_keys)
    
    def recursive_phi(keys):
        if not keys:
            return g(x0)
        current_key = keys[0]
        rest_keys = keys[1:]

        # Calculate g with all rest keys in x_plus and current_key in x0
        params_rest = {key: x_plus[key] if key not in keys else x0[key] for key in all_keys}
        phi_rest = recursive_phi(rest_keys)

        # Calculate g with all keys in x_plus
        params_all_plus = {key: x_plus[key] for key in all_keys}
        phi_all_plus = g(params_all_plus)

        # Calculate all lower order phi terms if rest_keys is not empty
        lower_order_phis = 0
        if len(rest_keys) > 0:
            lower_order_phis = sum(recursive_phi(subset) for subset in itertools.combinations(rest_keys, len(rest_keys) - 1))

        return phi_all_plus - phi_rest - lower_order_phis

    phi_terms = {}
    for i in range(1, n + 1):
        for subset in itertools.combinations(all_keys, i):
            phi_terms[subset] = recursive_phi(list(subset))

    return phi_terms

In [41]:
calculate_phi(x_plus, x0, g)

{('m',): Array(334.66406, dtype=float32, weak_type=True),
 ('c',): Array(334.66406, dtype=float32, weak_type=True),
 ('s',): Array(334.66406, dtype=float32, weak_type=True),
 ('m', 'c'): Array(0., dtype=float32, weak_type=True),
 ('m', 's'): Array(0., dtype=float32, weak_type=True),
 ('c', 's'): Array(0., dtype=float32, weak_type=True),
 ('m', 'c', 's'): Array(6521.963, dtype=float32, weak_type=True)}

Computational cost is $2^n$, which is not feasible as the number of inputs $n$ grows.

For a large number of inputs, the author recommends computing the first order ($\Phi_i$), total order ($\Phi_i^T$), and interaction ($\Phi_i^I$) sensitivity indices. This has a computational cost of $2n+2$. 

$$\Phi_i^T = \Phi_i + \sum_{j=1}^{n - 1}\Phi_{i,j} + ... + \Phi_{1,2,...,n}$$
$$\Phi_i^I = \Phi_i^T - \Phi_i$$

## Differentiation-based Methods


In [4]:
import jax 
import jax.numpy as jnp

df_dm = jax.grad(ooq, argnums=0)(x0['m'], x0['c'], x0['s'])
print(df_dm)

2.7872467


In [6]:
df_dc = jax.grad(ooq, argnums=1)(x0['m'], x0['c'], x0['s'])
print(df_dc)

-253949.12


In [7]:
df_ds = jax.grad(ooq, argnums=2)(x0['m'], x0['c'], x0['s'])
print(df_ds)

1594.5643


Partial derivatives themselves are not comparable, because they are denominated in different units. 

Thus, we define the Differential Importance Measure, $D_i$, which indicates the DIM of the variable $x_i$ in our input vector. 

The **elasticity** of $x_i$, denoted $E_i$, is an intermediate calculation. We multiply by $x_i^0$ to cancel out the difference in units between each $x_i$.

$$E_i = \frac{\partial g(x^0)}{\partial x_i} \frac{x_i^0}{g(x^0)}$$

$$D_i = \frac{E_i}{\sum_{j=1}^n E_j}$$

In [15]:
from typing import Callable

def elasticity(x: dict, func: Callable) -> float:
    y0 = func(x['m'], x['c'], x['s'])
    inputs = x['m'], x['c'], x['s']
    elasticities = []
    for i, input in enumerate(inputs):
        partial = jax.grad(func, argnums=i)(x['m'], x['c'], x['s'])
        elasticities.append(partial * input / y0)
    return jnp.array(elasticities)

In [17]:
elastics = elasticity(x0, ooq)

In [26]:
from jax.typing import ArrayLike

def diff_importance(elasticities: ArrayLike) -> ArrayLike:
    return jnp.round(elasticities / jnp.sum(elasticities), 3)

In [27]:
diff_importance(elastics)

Array([ 1., -1.,  1.], dtype=float32)

We find that the Differential Importance Measures of each variable have the same magnitude, with $x_2$ having an opposite effect as the other two inputs.

Additivity is a convenient property of DIM. We see that if we change $x_1$ and $x_2$ at the same rate, the effect on the output is 0.  